# 微调预训练模型

使用预训练模型有很多好处。它可以降低计算成本、减少碳足迹，并且让您可以使用最先进的模型，而无需从头开始训练一个。🤗 Transformers为各种任务提供了数千个预训练模型。当您使用预训练模型时，您会在特定于您任务的数据集上对其进行训练。这被称为微调，是一种非常强大的训练技术。在本教程中，您将使用您选择的深度学习框架对一个预训练模型进行微调：

- 使用🤗 Transformers Trainer微调预训练模型。
- 使用TensorFlow中的Keras微调预训练模型。
- 使用原生PyTorch微调预训练模型。

## 准备数据集

在微调预训练模型之前，下载一个数据集并准备好用于训练。之前的教程向您展示了如何处理训练数据，现在您有机会将这些技能付诸实践！

首先加载Yelp评论数据集：

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

正如您现在所知，您需要一个分词器来处理文本，并包括填充和截断策略以处理任何可变序列长度。为了一次处理整个数据集，请使用🤗 Datasets的map方法对整个数据集应用预处理函数：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

如果您愿意，您可以创建一个较小的子集来微调完整数据集，以减少所需的时间：

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 训练

此时，您应该按照您想要使用的框架对应的部分进行操作。您可以使用右侧边栏中的链接跳转到您想要的部分 - 如果您想隐藏特定框架的所有内容，只需使用该框架块右上角的按钮！

### 使用Keras训练TensorFlow模型
您也可以使用Keras API在TensorFlow中训练🤗 Transformers模型！

### 加载Keras的数据
当您想要使用Keras API训练🤗 Transformers模型时，您需要将数据集转换为Keras理解的格式。如果您的数据集很小，您可以将整个数据集转换为NumPy数组并将其传递给Keras。在做更复杂的事情之前，让我们先尝试这个。

首先，加载一个数据集。我们将使用GLUE基准测试中的CoLA数据集，因为它是一个简单的二元文本分类任务，现在只使用训练集。

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset = dataset["train"]  # Just take the training split for now

接下来，加载一个分词器，并将数据标记为NumPy数组。请注意，标签已经是一个由0和1组成的列表，因此我们可以直接将其转换为NumPy数组而无需进行标记化！

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
tokenized_data = tokenizer(dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])  # Label is already an array of 0 and 1

最后，加载、编译和拟合模型。请注意，Transformers模型都有一个默认的与任务相关的损失函数，因此除非您想要指定一个，否则不需要指定一个！

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.fit(tokenized_data, labels)

> 当您编译模型时，您不必传递损失参数！如果将此参数留空，Hugging Face模型会自动选择适合其任务和模型架构的损失。如果您想要，您始终可以通过指定自己的损失来覆盖这一点！

这种方法对于较小的数据集效果很好，但对于较大的数据集，您可能会发现开始出现问题。为什么？因为分词后的数组和标签必须完全加载到内存中，而且因为NumPy不处理“不规则”数组，所以每个分词样本都必须填充到整个数据集中最长样本的长度。这将使您的数组变得更大，并且所有这些填充标记也会减慢训练速度！

### 将数据加载为tf.data.Dataset

如果您想避免训练变慢，您可以将数据加载为tf.data.Dataset。虽然如果您愿意，您可以编写自己的tf.data流水线，但我们有两种方便的方法来实现这一点：

- prepare_tf_dataset()：这是我们在大多数情况下推荐的方法。因为它是您模型的一个方法，它可以检查模型以自动确定哪些列可用作模型输入，并丢弃其他列以创建一个更简单、性能更好的数据集。
- to_tf_dataset：这种方法更低级，当您想要精确控制数据集的创建方式时很有用，通过指定要包含的确切列和标签列。

在使用prepare_tf_dataset()之前，您需要将分词器的输出添加到数据集作为列，如下面的代码示例所示：

In [ ]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])


dataset = dataset.map(tokenize_dataset)

请记住，默认情况下，Hugging Face数据集存储在磁盘上，因此这不会增加您的内存使用！一旦列已添加，您可以从数据集中流式传输批次，并对每个批次进行填充，这将大大减少与对整个数据集进行填充相比的填充标记数量。

In [ ]:
tf_dataset = model.prepare_tf_dataset(dataset["train"], batch_size=16, shuffle=True, tokenizer=tokenizer)

请注意，在上面的代码示例中，您需要将分词器传递给prepare_tf_dataset，以便它可以在加载时正确填充批次。如果您的数据集中的所有样本长度相同且不需要填充，则可以跳过此参数。如果您需要执行比仅填充样本更复杂的操作（例如为掩码语言建模损坏标记），则可以使用collate_fn参数，以传递一个函数，该函数将被调用以将样本列表转换为批次并应用任何您想要的预处理。查看我们的示例或笔记本，以查看此方法的实际操作。

一旦创建了tf.data.Dataset，您可以像以前一样编译和拟合模型：

In [ ]:
model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.fit(tf_dataset)